In [12]:
import spacy
from spacy.tokens import Doc, Token

# Load model (tiếng Anh hoặc tiếng Việt đều được)
nlp = spacy.load("en_core_web_sm")  # hoặc "vi_core_news_lg" cho tiếng Việt

In [13]:
def find_main_verb(doc: Doc) -> Token | None:
    """
    Tìm động từ chính của câu (thường là token có dep_ == 'ROOT')
    Nếu ROOT không phải động từ thì trả về None hoặc tìm động từ gần nhất hợp lý
    """
    for token in doc:
        if token.dep_ == "ROOT":
            # Thông thường ROOT là động từ, nhưng đôi khi là danh từ (câu danh từ)
            if token.pos_ in ["VERB", "AUX"]:
                return token
            else:
                # Nếu ROOT không phải động từ, tìm động từ làm con trực tiếp
                for child in token.children:
                    if child.pos_ in ["VERB", "AUX"]:
                        return child
    return None

# Test
doc = nlp("The cat is sleeping peacefully on the sofa.")
main_verb = find_main_verb(doc)
print("Động từ chính:", main_verb)  # Output: sleeping

Động từ chính: sleeping


In [14]:
def my_noun_chunks(doc: Doc):
    """
    Trích xuất các cụm danh từ mà không dùng doc.noun_chunks
    Cách làm: Với mỗi danh từ (NOUN, PROPN), gom tất cả các từ bổ nghĩa
    nằm bên trái và thuộc subtree của nó (det, amod, compound, poss, etc.)
    """
    chunks = []

    for token in doc:
        # Chỉ xét các token là danh từ hoặc đại từ làm core của NP
        if token.pos_ not in ("NOUN", "PROPN", "PRON"):
            continue

        # Tìm biên trái: duyệt ngược về trái qua các dependency thuộc NP
        left = token
        while True:
            # Các con bên trái thường là det, amod, compound, poss, nummod,...
            left_children = [child for child in left.children
                           if child.i < left.i and child.dep_ in
                           ("det", "amod", "compound", "poss", "nummod", "clf", "case")]
            if not left_children:
                break
            # Lấy child xa nhất bên trái
            left = min(left_children, key=lambda t: t.i)

        # Tìm biên phải: các từ bên phải thuộc subtree nhưng vẫn trong NP
        right = token
        while True:
            right_children = [child for child in right.children
                            if child.i > right.i and child.dep_ in
                            ("compound", "amod", "nmod", "appos", "acl", "relcl")]
            if not right_children:
                break
            right = max(right_children, key=lambda t: t.i)

        start = left.i
        end = right.i + 1  # slice là [start:end)
        chunk_span = doc[start:end]

        # Tránh trùng lặp (do nhiều token cùng làm head)
        if not any(chunk_span.start >= c.end or chunk_span.end <= c.start for c in chunks):
            chunks.append(chunk_span)

    return chunks

# Test
doc = nlp("The quick brown fox jumps over the lazy dog.")
for chunk in my_noun_chunks(doc):
    print(chunk.text)
# Output:
# The quick brown fox
# the lazy dog

The quick brown fox


In [15]:
def get_path_to_root(token: Token):
    """
    Trả về danh sách các token từ token hiện tại lên đến ROOT (bao gồm cả hai)
    Ví dụ: token -> parent -> grandparent -> ... -> ROOT
    """
    path = []
    current = token
    while current is not None:
        path.append(current)
        if current.dep_ == "ROOT":
            break
        current = current.head

    return path

# Test
doc = nlp("I love eating delicious chocolate cake in the morning.")
token = doc[2]  # eating
path = get_path_to_root(token)

for t in path:
    print(f"{t.text} [{t.dep_}] <- ", end="")
print("ROOT")
# Output: eating [xcomp] <- love [ROOT] <- ROOT

eating [xcomp] <- love [ROOT] <- ROOT
